In [1]:
from google.colab import drive, output
drive.mount('/content/drive')
import sys
%cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
#sys.path.append('/content/drive/MyDrive/Deep Learning/AutoEncoders/Project/VQVAE_Working/data')
#sys.path.append('/content/drive/MyDrive/Deep Learning/AutoEncoders/Project/VQVAE_Working/models')
# sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Dataloader')
sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Models')
sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Modules')
sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Modules')
sys.path.append('/content/drive/Shareddrives/ConservacionBiologicaIA/Datos/Acceso_Datos_Humboldt/ensayo.xlsx')

%load_ext autoreload
%autoreload 1
!pip install torchaudio
!pip install optuna
!pip install wandb --upgrade
!wandb login
output.clear()

In [2]:
from __future__ import print_function
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import librosa
import librosa.display

from six.moves import xrange

#import umap
import wandb
import torch
torch.cuda.empty_cache()
import gc
gc.collect()
torch.cuda.empty_cache()


from scipy import signal
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim

from torch.utils.data import random_split
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import make_grid
import torch.nn.functional as F

#from ResidualStack import ResidualStack
#from Residual import Residual

#from Project.Modules.Dataloader import EcoDataTesisReduced
from Jaguas_DataLoader import SoundscapeData
#from Models import VAE

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = xm.xla_device()
print(device)

from datetime import timedelta
import wandb
from wandb import AlertLevel

import optuna

wandb.login()

cuda


wandb: Currently logged in as: danielnieto (use `wandb login --relogin` to force relogin)


True

In [3]:
def display_images(model_outputs, epoch):

    """
    Plots the original image and reconstructed image by the machine learning model. This function receives
    a list with a length of 3 like [epochs, original images, reconstructed images].

    Original images and reconstructed images have a shape of [num_images, num_channels, rows, columns].

    :param model_outputs: Outputs of the machine learning model.
    :param epoch: Desire epoch to visualize.
    :type epoch: int
    :return: matplotlib plot
    """

    recon = model_outputs[epoch][2].to("cpu").detach().numpy()
    img = model_outputs[epoch][1].to("cpu").detach().numpy()
    for index in range(recon.shape[0]):
        ax1 = plt.subplot(2, recon.shape[0], index+1)
        ax2 = plt.subplot(2, recon.shape[0], index+recon.shape[0]+1)

        librosa.display.specshow(librosa.power_to_db(recon[index, 0], ref=np.max),
                                 y_axis='mel', fmax=10000,
                                 x_axis='time', ax=ax1)

        librosa.display.specshow(librosa.power_to_db(img[index, 0], ref=np.max),
                                 y_axis='mel', fmax=10000,
                                 x_axis='time', ax=ax2)
    plt.show()

def plot_reconstructions(imgs_original, imgs_reconstruction, num_views: int=8):
    output = torch.cat((imgs_original[0:num_views], imgs_reconstruction[0:num_views]), 0)
    img_grid = make_grid(output, nrow=8, pad_value=20)
    fig, ax = plt.subplots(figsize=(20,5))
    ax.imshow(img_grid[1,:,:].cpu(), vmin=0, vmax=1)
    ax.axis("off")
    plt.show()
    return fig

def testModel(model, iterator):
    model.eval()
    (valid_originals, _,_) = next( iterator)
    valid_originals = torch.reshape(valid_originals, (valid_originals.shape[0] * valid_originals.shape[1], 
                                                  valid_originals.shape[2], valid_originals.shape[3]))
    valid_originals = torch.unsqueeze(valid_originals,1)

    valid_originals = valid_originals.to(device)

    vq_output_eval = model._pre_vq_conv(model._encoder(valid_originals))
    _, valid_quantize, _, _ = model._vq_vae(vq_output_eval)
    valid_reconstructions = model._decoder(valid_quantize)
    
    fig = plot_reconstructions(valid_originals, valid_reconstructions, 8)

    recon_error = F.mse_loss(valid_originals, valid_reconstructions)

    return fig, recon_error

def define_encoder(trial):
    n_layers = trial.suggest_int("n_layers", 1, 5)
    layers = []

    in_features = 1
    for i in range(n_layers):
        #out_features = i+1**2
        out_features = trial.suggest_int("n_units_l{}".format(i), (i+2)**3, 2*(i+2)**3, log=True)
        layers.append(nn.Conv2d(in_channels=in_features,
                                out_channels=out_features,
                                kernel_size=(4, 4),
                                padding=(0, 0),
                                stride=(1, 1)
                                )
                      )
        if i != n_layers - 1:
            layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_{}".format(i), 0.2, 0.5)
        #layers.append(nn.Dropout(p))
        encoder = layers
        in_features = out_features

    return nn.Sequential(*encoder)


def define_decoder(trial):

    decoder = []
    encoder = define_encoder(trial)
    decoder_counter = 0
    for i in range(len(encoder) - 1, -1, -1):
        decoder.append(encoder[i])
        if type(decoder[decoder_counter]) == torch.nn.modules.conv.Conv2d:
            decoder[decoder_counter] = nn.ConvTranspose2d(in_channels=encoder[i].out_channels,
                                                          out_channels=encoder[i].in_channels,
                                                          kernel_size=encoder[i].kernel_size,
                                                          stride=encoder[i].stride,
                                                          padding=encoder[i].padding)
        decoder_counter += 1

    decoder.append(nn.ReLU())
    decoder.append(nn.Sigmoid())
    decoder = nn.Sequential(*decoder)

    return decoder


class define_model(nn.Module):
    def __init__(self, trial):
        super().__init__()
        self.encoder = define_encoder(trial)
        self.decoder = define_decoder(trial)
        #self.autoencoder = define_autoencoder(trial)

    def forward(self, x):

        encoded = self.encoder(x)
        # print(f"shape {encoded.shape}" )
        decoder = self.decoder(encoded)

        return decoder

def train_model(model, optimizer, num_images, dataloader):

    model.train()
    criterion = nn.MSELoss()

    i = 0
    for (img, _, _) in dataloader:
        print(f"i{i}")
        img = img.reshape(-1, img.shape[2], img.shape[3])
        img = torch.unsqueeze(img, 1)

        if i >= num_images:
            break

        img_cuda = img.to(device)
        recon = model(img_cuda)
        loss = criterion(recon, img_cuda)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i += 1

        wandb.log({"loss":loss.item()}
                 )
    return loss, img, recon

def objective(trial):

    config = {
    "batch_size" : 20,
    "num_epochs": 15,
    # "num_training_updates" : len(dataset_train),
    "num_hiddens" : 64,
    "embedding_dim" : 128,
    "zDim" : 128,
    "commitment_cost" : 0.25,
    "decay" : 0.99,
    "learning_rate" : 1e-3,
    "dataset": "Audios Jaguas",
    "architecture": "AE",
              }

    wandb.config = config
    num_images = 2000

    print("-----------Running-----------")
    root_path = '/content/drive/Shareddrives/ConservacionBiologicaIA/Datos/Jaguas_2018'
    
    dataset = SoundscapeData(root_path, 1, "wav")
    dataset_train, dataset_test = random_split(dataset,
                                                [round(len(dataset)*0.10),
                                                 len(dataset) - round(len(dataset)*0.10)], 
                                                generator=torch.Generator().manual_seed(1024))
    training_loader = DataLoader(dataset_train, batch_size=config["batch_size"], shuffle = False)
    test_loader = DataLoader(dataset_test, batch_size=config["batch_size"])

    model = define_model(trial).to(device)
    optimizer = torch.optim.Adam(
        model.parameters(), trial.suggest_float("lr", 1e-5, 1e-1, log=True))
    
    
    wandb.init(project="AE-Jaguas", config=config)
    wandb.watch(model, F.mse_loss, log="all", log_freq=1)

    outputs = []

    for epoch in range(config['num_epochs']):
        loss, img, recon= train_model(model=model, optimizer=optimizer, num_images=num_images, dataloader=training_loader)
        recon = recon.to('cpu')
        print(f'Epoch:{epoch + 1}, Loss: {loss.item():.4f}')
        outputs.append((epoch, img, recon))
        fig = plot_reconstructions(img,recon)   
        # display_images(outputs, epoch)
        images = wandb.Image(fig, caption= f"recon_error: {np.round(loss.item(),4)}")
        wandb.log({"examples": images})

    return loss

In [4]:
wandb.finish()
study = optuna.create_study(directions=["minimize"])
study.optimize(objective, n_trials=5, timeout=300)

[I 2022-04-16 22:21:17,742] A new study created in memory with name: no-name-6fb73c16-cdf0-4c15-a687-5a1fde73fe99


-----------Running-----------


i0


[W 2022-04-16 22:21:33,991] Trial 0 failed because of the following error: RuntimeError('CUDA out of memory. Tried to allocate 2.39 GiB (GPU 0; 15.90 GiB total capacity; 13.02 GiB already allocated; 1.88 GiB free; 13.03 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-3-6666988fe913>", line 187, in objective
    loss, img, recon= train_model(model=model, optimizer=optimizer, num_images=num_images, dataloader=training_loader)
  File "<ipython-input-3-6666988fe913>", line 135, in train_model
    recon = model(img_cuda)
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 1120, in _call_impl
    result = forward_call(*input,

RuntimeError: ignored

In [ ]:
root_path = '/content/drive/Shareddrives/ConservacionBiologicaIA/Datos/Jaguas_2018'


dataset = SoundscapeData(root_path, 1, "wav")
dataset_train, dataset_test = random_split(dataset,
                                           [round(len(dataset)*0.0250), len(dataset) - round(len(dataset)*0.0250)], 
                                           generator=torch.Generator().manual_seed(1024))

config = {
    "batch_size" : 20,
    "num_epochs": 15,
    "num_training_updates" : len(dataset_train),
    "num_hiddens" : 64,
    "embedding_dim" : 128,
    "zDim" : 128,
    "commitment_cost" : 0.25,
    "decay" : 0.99,
    "learning_rate" : 1e-3,
    "dataset": "Audios Jaguas",
    "architecture": "VQ-VAE",
}

training_loader = DataLoader(dataset_train, batch_size=config["batch_size"])
test_loader = DataLoader(dataset_test, batch_size=config["batch_size"])


model = VAE(num_hiddens=config["num_hiddens"],
              zDim=config["zDim"]).to(device)

optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"], amsgrad=False)


wandb.finish()
wandb.init(project="VAE-Jaguas", config=config)
wandb.watch(model)


In [ ]:
def plot_reconstructions(imgs_original, imgs_reconstruction, num_views: int=8):
    output = torch.cat((imgs_original[0:num_views], imgs_reconstruction[0:num_views]), 0)
    img_grid = make_grid(output, nrow=8, pad_value=20)
    fig, ax = plt.subplots(figsize=(20,5))
    ax.imshow(img_grid[1,:,:].cpu(), vmin=0, vmax=1)
    ax.axis("off")
    plt.show()

def testModel(model, iterator):
    model.eval()
    (valid_originals, _,_) = next( iterator)
    valid_originals = torch.reshape(valid_originals, (valid_originals.shape[0] * valid_originals.shape[1], 
                                                  valid_originals.shape[2], valid_originals.shape[3]))
    valid_originals = torch.unsqueeze(valid_originals,1)

    valid_originals = valid_originals.to(device)

    mu, logvar = model.encoder(valid_originals)
    z = model.reparameterize(mu, logvar)
    valid_reconstructions = model.decoder(z)
    plot_reconstructions(valid_originals, valid_reconstructions)
    recon_error = F.mse_loss(valid_originals, valid_reconstructions)
    return fig, recon_error

In [ ]:
model.train()
train_res_recon_error = []
train_res_perplexity = []
iterator = iter(test_loader)
wandb.watch(model, F.mse_loss, log="all", log_freq=1)
error_files = []

for epoch in range(config["num_epochs"]):
  for i in xrange(config["num_training_updates"]):
      model.train()
      try:
         (data, _,_) = next(iter(training_loader))
      except:
        error_files.append[i]
        continue
          
      data = torch.reshape(data, (data.shape[0] * data.shape[1], data.shape[2], data.shape[3]))
      data = torch.unsqueeze(data,1)
      data = data.to(device)
      print(data.shape)
      optimizer.zero_grad()
      data_recon, mu, logvar = model(data)
      print(data_recon.shape)
      
      recon_error = F.mse_loss(data_recon, data) #/ data_variance
      loss = recon_error
      loss.backward()

      optimizer.step()

      print(f'epoch: {epoch} of {config["num_epochs"]} \t iteration: {(i+1)} of {config["num_training_updates"]} \t loss: {np.round(loss.item(),4)}')

      if (i+1) % 5 == 0:
        #torch.save(model.state_dict(),f'model_{epoch}_{i}.pkl')
        fig, test_error = testModel(model, iterator)
        images = wandb.Image(fig, caption= f"recon_error: {np.round(test_error.item(),4)}")
        wandb.log({"examples": images})

wandb.finish()